<center><h1>Informarion Retrival Project</h1></center>
<center><h1>Build an Intelligent Information Retrival System</h1></center>

## Importation des librairies et du dataset

In [18]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import re
import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../dataset_dbscan.csv")
df.drop(columns = df.columns[0], axis = 1, inplace= True)
df

,18,told,biographi,briefli,describ,system,attempt,provid,decim,detail,...,anglo,supposit,morn,claus,inventor,sell,patente,monopoli,15%,9%
0,0.565909,0.64087,0.672031,0.410718,0.201332,0.145940,0.275780,0.209136,0.982483,0.29727,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.116752,0.000000,0.167309,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.000000,0.00000,0.000000,0.000000,0.000000,0.233504,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1456,0.000000,0.00000,0.000000,0.000000,0.000000,0.097293,0.183853,0.000000,0.000000,0.00000,...,0.527442,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1457,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,1.054883,1.054883,1.054883,1.054883,1.054883,1.054883,1.054883,0.000000,0.000000
1458,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Clusterisation des documents avec DBSCAN

In [3]:
# Retourner tous les points du dataset qui sont à une distance inférieure à eps de notre instance
def region_query(distances, i, eps):
    doc_indexes = np.where(distances[i] <= eps)[0] # On récupère les index des points qui sont à une distance inférieure à eps
    return [i for i in doc_indexes] # On retourne la liste des index des points

# Fonction qui permet de récursivement ajouter les points voisins à notre cluster
def expand_cluster(distances, document_cluster, clusters, C, neighbors, eps, min_pts):
    while len(neighbors)!=0: # Tant qu'il y a des voisins
        index_doc = neighbors.pop() # On récupère le premier voisin
        if document_cluster[index_doc] >=0 : continue # Si le point appartient déjà à un cluster et n'est pas du bruit, on passe au suivant
        document_cluster[index_doc] = C # On change la valeur -1 par le numéro du cluster pour indiquer que le point appartient à ce cluster
        clusters[C].append(index_doc) # On ajoute le point au cluster
        new_neighbors = region_query(distances,index_doc,eps) # On récupère les points voisins du point
        if len(new_neighbors)>= min_pts: neighbors.extend(new_neighbors) # Si le point a assez de voisins, on ajoute les voisins au cluster

# Fonction recréant l'algorithme DBSCAN
def dbscan(distances, eps, min_pts):
    # On itilisalise une liste a -1 pour chaque point du dataset (on ne connait pas encore son cluster)
    document_cluster = np.full(len(distances), (-1), dtype = np.int)
    clusters = [[]]
    noise = []
    C = 0
    for index_doc in range(len(distances)):
        if document_cluster[index_doc] != -1 : continue # Si le point appartient déjà à un cluster, on passe au suivant
        
        neighbors = region_query(distances,index_doc,eps) # On récupère les points voisins
        
        if len(neighbors) >= min_pts: # Si le point a assez de voisins, on crée un nouveau cluster
            document_cluster[index_doc] = C # On change la valeur -1 par le numéro du cluster pour indiquer que le point appartient à ce cluster
            clusters[C].append(index_doc) # On ajoute le point au cluster
            expand_cluster(distances, document_cluster, clusters, C, neighbors, eps, min_pts) # On ajoute les voisins du point au cluster
            C += 1 # On passe au cluster suivant
            clusters.append([])  # On initialise le nouveau cluster
        else : # Si le point n'a pas assez de voisins, on le considère comme du bruit
            document_cluster[index_doc] = -2 # On change la valeur -1 par -2 pour indiquer que le point est du bruit
            noise.append(index_doc) # On ajoute le point au bruit

            
    del clusters[C] # On supprime le dernier cluster qui est vide
    
    return clusters, noise

In [4]:
# Calculer la distance de manhattan entre tout les points du dataset
# Pour économiser du temps d'execution, on ne calcule que la matrice superieure de la matrice de distance
def manhattan_distance(df):
    distances = np.zeros((len(df), len(df)))
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            dist = np.sum(np.abs(df[i] - df[j]))
            
            distances[i][j] = dist
            distances[j][i] = dist

    return distances

# Fonction qui permet de calculer la distance de manhattan entre deux vecteurs en prenant en compte les valeurs communes
def manhattan_distance_with_coeff(df):
    distances = np.zeros((len(df), len(df))) # On initialise la matrice de distance
    for i in range(len(df)): # On parcourt tous les points du dataset
        for j in range(i+1, len(df)): # On parcourt tous les points du dataset
            
            index_a = np.where(df[i]>0)[0] # On récupère les index des valeurs non nulles du vecteur a
            index_b = np.where(df[j]>0)[0] # On récupère les index des valeurs non nulles du vecteur b
            common = np.intersect1d(index_a, index_b) # On récupère les index des valeurs communes
            
            # On multiplie par 5 le poid des valeurs non communes (Pour éloigner les documents avec beaucoup de valeurs non communes)
            new_a = df[i] * 5 
            new_b = df[j] * 5
            
            # On divise par 100 le poid des valeurs communes (Pour rapprocher les documents avec beaucoup de valeurs communes)
            new_a[common] = df[i][common] * 0.01
            new_b[common] = df[j][common] * 0.01
            
            dist = np.sum(np.abs(new_a - new_b)) # On calcule la distance de manhattan
            
            # On divise par le nombre de valeurs communes si il y en a plus que de valeurs non communes (Toujours pour rapprocher les documents avec beaucoup de valeurs communes)
            if len(common) > len(df[i]) - len(common): dist = dist / len(common) 
            else: dist = dist * (len(df[i]) - len(common)) # On multiplie par le nombre de valeurs non communes si il y en a plus que de valeurs communes (Toujours pour éloigner les documents avec beaucoup de valeurs non communes)
            
            # On met la distance entre le document i et le document j et la distance entre le document j et le document i à la même valeur
            distances[i][j] = dist 
            distances[j][i] = dist 

    return distances

In [7]:
def calcule_distances(df, methode):
    if methode == "manhattan": return manhattan_distance(df)
    elif methode == "manhattan_with_coeff": return manhattan_distance_with_coeff(df)
distances = calcule_distances(df.to_numpy(), methode = "manhattan")

In [10]:
clusters = dbscan(distances, 50, 3)
len(clusters), len(clusters[0]), len(clusters[1])

(2, 1, 38)

## DBSCAN après nettoyage et réduction des dimmensions

In [11]:
number_cols = [x for x in df.columns if re.search(r'\d', x)] # On récupère les colonnes contenant des chiffres
percent_num = 100 * len(number_cols) / len(df.columns) # On calcule le pourcentage de colonnes contenant des chiffres

print(f"{percent_num:.2f}% of the columns are numbers") # On affiche le pourcentage de colonnes contenant des chiffres

7.07% of the columns are numbers


Les valeurs numérique représentant souvent un problème lors du clustering en text mining nous allons les supprimer

In [12]:
df_clean = df.copy() # On copie le dataset
for c in number_cols:
    df_clean.drop(c, axis=1, inplace=True)
df.shape, df_clean.shape

((1460, 6929), (1460, 6439))

In [13]:
# On applique PCA pour la réduction de dimension
def pca_decomp(df, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(df)
    df_pca = pca.transform(df)
    df_pca = pd.DataFrame(df_pca)
    return pca, df_pca

In [21]:
pca, df_pca = pca_decomp(df_clean, 20)
df_pca

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.162252,-0.034010,-0.054608,0.016541,-0.112128,0.331772,-0.200726,-0.109657,0.075522,-0.200862,0.034117,-0.358309,-0.423753,-0.002277,0.225348,-0.117654,-0.207261,-0.116705,-0.108271,-0.001785
1,-0.179240,-0.030959,0.117689,-0.068736,-0.050065,-0.123451,-0.086458,0.107275,-0.149041,0.068116,0.021445,-0.096053,0.013292,0.124328,0.166495,0.051489,-0.078320,-0.021788,0.040126,0.110348
2,-0.097213,0.028530,-0.095813,0.074921,-0.105212,-0.038557,-0.082638,-0.083539,0.188612,-0.168050,-0.135951,0.222267,0.210461,0.270871,0.180987,0.013752,0.006380,-0.253531,-0.186246,-0.199515
3,-0.334536,-0.073481,0.337738,0.014843,-0.086564,-0.063403,-0.097596,0.125289,-0.155686,0.144950,0.207175,-0.055303,-0.078722,-0.060767,-0.051402,-0.121151,-0.107249,-0.020947,0.114089,-0.093923
4,-0.181552,-0.001350,0.005224,0.157100,0.019773,-0.068349,-0.048160,-0.050190,-0.060843,0.041224,0.183442,-0.176355,0.075970,-0.236173,-0.060067,-0.221625,0.181336,-0.196345,0.329191,0.101396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.251164,-0.021111,-0.182953,0.062115,-0.080617,-0.062221,-0.015455,-0.108317,0.004804,-0.000861,-0.022488,0.065298,0.210294,0.184807,0.006527,-0.095116,0.073930,-0.108577,0.029812,0.127263
1456,-0.468549,-0.018110,0.207495,0.042913,0.048822,-0.001121,-0.166097,-0.014878,-0.108164,0.103044,0.217450,-0.057605,-0.026400,0.138455,-0.012691,-0.088828,-0.147955,-0.169988,0.035414,0.045271
1457,-0.312563,0.016691,-0.330556,0.064363,-0.078879,-0.047549,-0.069160,-0.066980,0.170529,-0.166054,-0.013644,0.229184,-0.376994,-0.017905,-0.002527,-0.151284,0.059661,-0.456854,-0.507894,0.203570
1458,0.075373,0.011343,-0.176032,0.087835,-0.238684,0.137505,-0.185389,0.238832,0.131386,0.122301,0.064455,0.005650,0.106323,0.111902,-0.017246,0.274239,-0.003908,-0.172776,0.050760,0.034902


In [ ]:
plt.bar(range(pca.n_components_), pca.explained_variance_ratio_)
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [22]:
distances = calcule_distances(df_pca.to_numpy(), methode = "manhattan")

In [25]:
clusters = dbscan(distances, 4, 3)
len(clusters), len(clusters[0]), len(clusters[1])

(2, 1, 40)